In [1]:
import plotly.graph_objects as go
import numpy as np
from tqdm.auto import tqdm

In [2]:
def unit_sphere():
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x = np.outer(np.cos(u), np.sin(v))
    y = np.outer(np.sin(u), np.sin(v))
    z = np.outer(np.ones(np.size(u)), np.cos(v))

    sphere = go.Surface(
        x=x,
        y=y,
        z=z,
        opacity=0.1,
        showscale=False,
        colorscale=[[0, 'blue'], [1, 'blue']]
    )
    return sphere 

def regular_tetrahron(k=0.1):
    # Define vertices
    vertices = k * np.array([
        [1, 1, 1],    # vertex 1
        [1, -1, -1],  # vertex 2 
        [-1, 1, -1],  # vertex 3
        [-1, -1, 1]   # vertex 4
    ])

    # Create scatter3d trace for vertices
    scatter = go.Scatter3d(
        x=vertices[:,0],
        y=vertices[:,1], 
        z=vertices[:,2],
        mode='markers+text',
        marker=dict(
            size=5,  # Reduced dot size from 10 to 5
            color='blue'
        ),
        text=['1', '2', '3', '4'],
        textposition='top center'
    )
    return scatter 

In [3]:
fig = go.Figure(data=[unit_sphere()])
fig.add_trace(regular_tetrahron(.4))

# Update layout
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y', 
        zaxis_title='Z',
        aspectmode='cube'
    ),
    width=800,
    height=800,
    showlegend=False
)

fig.show()

In [25]:
import pyvista as pv
import numpy as np
import plotly.graph_objects as go
from hyperbolic3d.gyro import gyrotriangle

def pv_to_npmesh(pv_mesh):
    vertices = pv_mesh.points
    faces = pv_mesh.faces.reshape(-1, 4)[:, 1:4]
    print(vertices.shape, faces.shape)
    return {'V': vertices, 'F': faces}

class RegularTetrahedron:
    def __init__(self, r=.5, name=None):
        r = 0.5  # Radius from center (must be < 1)
        self.vertices = r * np.array([
                [1, 1, 1],    # vertex 1
                [1, -1, -1],  # vertex 2 
                [-1, 1, -1],  # vertex 3
                [-1, -1, 1]   # vertex 4
            ])
        self.name = name 
        faces = [
            [0, 1, 2],
            [0, 1, 3],
            [0, 2, 3],
            [1, 2, 3]
        ]
        self.face_meshes = {}
        for face in faces:
            face_id = list(set((0, 1, 2, 3)) - set(face))[0]
            self.face_meshes[face_id] = pv_to_npmesh(gyrotriangle(
                self.vertices[face[0]], 
                self.vertices[face[1]], 
                self.vertices[face[2]], 
                s=1., 
                depth=6
            ))

    def plot(self, color='blue'):
        traces = []
        
        # Add face meshes
        for face_id, mesh in self.face_meshes.items():
            vertices = mesh['V']
            faces = mesh['F']
            traces.append(go.Mesh3d(
                x=vertices[:, 0],
                y=vertices[:, 1],
                z=vertices[:, 2],
                i=faces[:, 0],
                j=faces[:, 1],
                k=faces[:, 2],
                color=color,
                opacity=0.3
            ))
        
        # Add vertices with labels
        vertex_labels = []
        for i in range(len(self.vertices)):
            if self.name:
                label = f"({self.name}, {i})"
            else:
                label = f"({i})"
            vertex_labels.append(label)
        
        traces.append(go.Scatter3d(
            x=self.vertices[:,0],
            y=self.vertices[:,1],
            z=self.vertices[:,2],
            mode='markers+text',
            marker=dict(
                size=5,
                color=color
            ),
            text=vertex_labels,
            textposition='top center'
        ))
        return traces

T = RegularTetrahedron(name="0", r=.5)
fig = go.Figure()
for trace in T.plot():
    fig.add_trace(trace)
fig.show()

(2145, 3) (4096, 3)
(2145, 3) (4096, 3)
(2145, 3) (4096, 3)
(2145, 3) (4096, 3)
